# Kyoto Protocol, world performance analysis

## Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

## 1. Data wrangling
### 1st Dataset - (1)mean exposure levels to PM2.5 by country and (2)percentage of people exposed to different air pollution levels by country

In [2]:
#Read 1st dataframe containing the air pollution levels per country
air_pollution_raw = pd.read_csv("PM2_5_pollution_exposure.csv")
air_pollution_raw.head(3)

,Country,Macroregion,Microregion,Variable,Year,Unit,PowerCode,Reference Period,Value,Flags
0,Australia,-Total-,-Total-,Mean population exposure to PM2.5,1990,Micrograms per cubic metre,Units,NaN,10.36402,NaN
1,Australia,-Total-,-Total-,Mean population exposure to PM2.5,2010,Micrograms per cubic metre,Units,NaN,10.53559,NaN
2,Australia,-Total-,-Total-,Mean population exposure to PM2.5,2011,Micrograms per cubic metre,Units,NaN,10.88267,NaN


In [3]:
#Remove useless columns
clean_air_pollution = air_pollution_raw.drop(["Macroregion", "Microregion", "PowerCode",
                                                 "Reference Period", "Flags"], axis = 1)
print(len(clean_air_pollution))
clean_air_pollution.head(10)

7352


,Country,Variable,Year,Unit,Value
0,Australia,Mean population exposure to PM2.5,1990,Micrograms per cubic metre,10.36402
1,Australia,Mean population exposure to PM2.5,2010,Micrograms per cubic metre,10.53559
2,Australia,Mean population exposure to PM2.5,2011,Micrograms per cubic metre,10.88267
3,Australia,Mean population exposure to PM2.5,2012,Micrograms per cubic metre,10.40148
4,Australia,Mean population exposure to PM2.5,2013,Micrograms per cubic metre,9.84329
5,Australia,Mean population exposure to PM2.5,2014,Micrograms per cubic metre,9.36117
6,Australia,Mean population exposure to PM2.5,2015,Micrograms per cubic metre,9.20277
7,Australia,Mean population exposure to PM2.5,2016,Micrograms per cubic metre,8.53581
8,Australia,Percentage of population exposed to more than ...,1990,Percentage,32.18407
9,Australia,Percentage of population exposed to more than ...,2010,Percentage,34.08280


In [4]:
#Check how many variable names are contained in "Variable" column
clean_air_pollution["Variable"].value_counts()

Percentage of population exposed to more than 15 micrograms/m3    1472
Percentage of population exposed to more than 10 micrograms/m3    1472
Percentage of population exposed to more than 25 micrograms/m3    1472
Percentage of population exposed to more than 35 micrograms/m3    1472
Mean population exposure to PM2.5                                 1464
Name: Variable, dtype: int64

In [5]:
clean_air_pollution["Country"].value_counts()

Saint Vincent and the Grenadines    40
Uzbekistan                          40
Solomon Islands                     40
Greece                              40
Venezuela                           40
                                    ..
Vanuatu                             40
Spain                               40
United Arab Emirates                40
Belize                              40
Cayman Islands                      32
Name: Country, Length: 184, dtype: int64

There are 8 missing values from the __"Mean population exposure to PM2.5"__ variable, all of them coming from __"Cayman Islands"__. Due to its low potential significance in its contribution to the world air pollution, we decided to get rid of the entire record for this country. 

In [6]:
#Remove Cayman Islands from dataframe
cayman = clean_air_pollution.loc[clean_air_pollution["Country"] == "Cayman Islands"].index
cleaner_air_pollution = clean_air_pollution.drop(cayman, axis = 0)

In [7]:
#Raname variable and unit names to make it easier to read
cleaner_air_pollution = cleaner_air_pollution.replace({"Variable" :
                                                 {"Mean population exposure to PM2.5": "Mean exposure level to PM2.5",
                                                 "Percentage of population exposed to more than 10 micrograms/m3" : "Pop exposed > 10µg/m3",
                                                 "Percentage of population exposed to more than 15 micrograms/m3" : "Pop exposed > 15µg/m3",
                                                 "Percentage of population exposed to more than 25 micrograms/m3" : "Pop exposed > 25µg/m3",
                                                 "Percentage of population exposed to more than 35 micrograms/m3" : "Pop exposed > 35µg/m3"}})

cleaner_air_pollution = cleaner_air_pollution.replace({"Unit" : {"Micrograms per cubic metre" : "µg/m3"}})
cleaner_air_pollution

,Country,Variable,Year,Unit,Value
0,Australia,Mean exposure level to PM2.5,1990,µg/m3,10.36402
1,Australia,Mean exposure level to PM2.5,2010,µg/m3,10.53559
2,Australia,Mean exposure level to PM2.5,2011,µg/m3,10.88267
3,Australia,Mean exposure level to PM2.5,2012,µg/m3,10.40148
4,Australia,Mean exposure level to PM2.5,2013,µg/m3,9.84329
...,...,...,...,...,...
7331,Iran,Mean exposure level to PM2.5,2012,µg/m3,38.80363
7332,Iran,Mean exposure level to PM2.5,2013,µg/m3,38.42392
7333,Iran,Mean exposure level to PM2.5,2014,µg/m3,38.27532
7334,Iran,Mean exposure level to PM2.5,2015,µg/m3,39.97714


In [8]:
#Create an independent dataframe containing only the mean exposure level per country
mean_air_pollution = cleaner_air_pollution.copy()
index_mean = mean_air_pollution.loc[cleaner_air_pollution["Variable"] != "Mean exposure level to PM2.5"].index
mean_air_pollution = mean_air_pollution.drop(index_mean, axis = 0)
print(len(mean_air_pollution))
mean_air_pollution.head()

1464


,Country,Variable,Year,Unit,Value
0,Australia,Mean exposure level to PM2.5,1990,µg/m3,10.36402
1,Australia,Mean exposure level to PM2.5,2010,µg/m3,10.53559
2,Australia,Mean exposure level to PM2.5,2011,µg/m3,10.88267
3,Australia,Mean exposure level to PM2.5,2012,µg/m3,10.40148
4,Australia,Mean exposure level to PM2.5,2013,µg/m3,9.84329


### 2nd Dataset - Deaths due to air pollution diseases by country

In [9]:
#Read 2nd dataframe containing the deaths due to air pollution diseases
deaths_raw = pd.read_csv("death_rates_air_pollution.csv")
print(len(deaths_raw))
deaths_raw

6468


,Entity,Code,Year,"Deaths - Air pollution - Sex: Both - Age: Age-standardized (Rate) (deaths per 100,000)"
0,Afghanistan,AFG,1990,299.477309
1,Afghanistan,AFG,1991,291.277967
2,Afghanistan,AFG,1992,278.963056
3,Afghanistan,AFG,1993,278.790815
4,Afghanistan,AFG,1994,287.162923
...,...,...,...,...
6463,Zimbabwe,ZWE,2013,143.850145
6464,Zimbabwe,ZWE,2014,138.200536
6465,Zimbabwe,ZWE,2015,132.752553
6466,Zimbabwe,ZWE,2016,128.692138


In [10]:
#Rename column name for convenience
deaths_raw.rename(columns = {"Deaths - Air pollution - Sex: Both - Age: Age-standardized (Rate) (deaths per 100,000)" :
                             "Deaths per 100,000"}, inplace = True)
deaths_raw

,Entity,Code,Year,"Deaths per 100,000"
0,Afghanistan,AFG,1990,299.477309
1,Afghanistan,AFG,1991,291.277967
2,Afghanistan,AFG,1992,278.963056
3,Afghanistan,AFG,1993,278.790815
4,Afghanistan,AFG,1994,287.162923
...,...,...,...,...
6463,Zimbabwe,ZWE,2013,143.850145
6464,Zimbabwe,ZWE,2014,138.200536
6465,Zimbabwe,ZWE,2015,132.752553
6466,Zimbabwe,ZWE,2016,128.692138


In [11]:
#Merge mean_air_pollution and deaths_raw dataframes
levels_deaths_df = mean_air_pollution.merge(deaths_raw, left_on = ["Country", "Year"], right_on = ["Entity", "Year"])
levels_deaths_df

,Country,Variable,Year,Unit,Value,Entity,Code,"Deaths per 100,000"
0,Australia,Mean exposure level to PM2.5,1990,µg/m3,10.36402,Australia,AUS,26.705027
1,Australia,Mean exposure level to PM2.5,2010,µg/m3,10.53559,Australia,AUS,13.571713
2,Australia,Mean exposure level to PM2.5,2011,µg/m3,10.88267,Australia,AUS,13.727626
3,Australia,Mean exposure level to PM2.5,2012,µg/m3,10.40148,Australia,AUS,12.659725
4,Australia,Mean exposure level to PM2.5,2013,µg/m3,9.84329,Australia,AUS,11.874490
...,...,...,...,...,...,...,...,...
1315,Iran,Mean exposure level to PM2.5,2012,µg/m3,38.80363,Iran,IRN,46.800381
1316,Iran,Mean exposure level to PM2.5,2013,µg/m3,38.42392,Iran,IRN,46.328081
1317,Iran,Mean exposure level to PM2.5,2014,µg/m3,38.27532,Iran,IRN,46.446436
1318,Iran,Mean exposure level to PM2.5,2015,µg/m3,39.97714,Iran,IRN,47.209842


In [12]:
#Rename "Value" column to be able to remove non-esencial columns
levels_deaths_df.rename(columns = {"Value" :
                             "Mean exposure toPM2.5 (µg/m3)"}, inplace = True)

air_poll = levels_deaths_df.drop(["Variable", "Unit", "Entity","Code"], axis = 1)
air_poll.head()

,Country,Year,Mean exposure toPM2.5 (µg/m3),"Deaths per 100,000"
0,Australia,1990,10.36402,26.705027
1,Australia,2010,10.53559,13.571713
2,Australia,2011,10.88267,13.727626
3,Australia,2012,10.40148,12.659725
4,Australia,2013,9.84329,11.874490


### New column - Status of ratification

In [13]:
ratified_countries = ["Albania", "Algeria", "Angola", "Antigua and Barbuda", "Argentina",
                     "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas",
                     "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
                     "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina",
                     "Botswana", "Brazil", "Bulgaria", "Burkina Faso", "Burundi",
                     "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic",
                     "Chile", "China", "Colombia", "Comoros", "Congo",
                     "Cook Islands", "Costa Rica", "Cote D\'Ivoire", "Croatia", "Cuba",
                     "Cyprus", "Czech Republic", "Democratic Peoples Republic of Korea", "Denmark", "Djibouti",
                     "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador",
                     "Equatorial Guinea", "Eritrea", "Estonia", "Ethiopia", "Fiji",
                     "Finland", "France", "Gabon", "Gambia", "Georgia",
                     "Germany", "Ghana", "Greece", "Grenada", "Guatemala",
                     "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras",
                     "Hungary", "Iceland", "India", "Indonesia", "Iran",
                     "Ireland", "Israel", "Italy", "Jamaica", "Japan",
                     "Jordan", "Kenya", "Kiribati", "Kuwait",
                     "Kyrgyzstan", "Lao Democratic Republic", "Latvia", "Lebanon", "Lesotho",
                     "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg",
                     "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali",
                     "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico",
                     "Micronesia", "Monaco", "Mongolia", "Montenegro", "Morocco",
                     "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal",
                     "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria",
                     "Niue", "Norway", "Oman", "Pakistan", "Palau", "Panama",
                     "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland",
                     "Portugal", "Qatar", "Republic of Korea", "Republic of Moldova", "Romania",
                     "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines",
                     "Samoa", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Serbia",
                     "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia",
                     "Solomon Islands", "South Africa", "Spain", "Sri Lanka", "Sudan",
                     "Suriname", "Swaziland", "Sweden", "Switzerland", "Syrian Arab Republic",
                     "Tajikistan", "Thailand", "Macedonia", "Timor-Leste", "Togo",
                     "Tonga", "Trinidad and Tobago", "Tunisia", "Turkmenistan", "Tuvalu",
                     "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "Tanzania",
                     "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam",
                     "Yemen", "Zambia"]

In [14]:
#Add a column "Ratified" with the status of ratification of the Kyoto Protocol for each country 
#air_poll["Ratified Kyoto P."] = ""

for index, row in air_poll.iterrows():
    if row["Country"] in ratified_countries:
        air_poll.loc[index, 'Ratified Kyoto P.'] = True
        
    else:
        air_poll.loc[index, 'Ratified Kyoto P.'] = False
        
air_poll.head()

,Country,Year,Mean exposure toPM2.5 (µg/m3),"Deaths per 100,000",Ratified Kyoto P.
0,Australia,1990,10.36402,26.705027,True
1,Australia,2010,10.53559,13.571713,True
2,Australia,2011,10.88267,13.727626,True
3,Australia,2012,10.40148,12.659725,True
4,Australia,2013,9.84329,11.874490,True


### 4th Dataset - Coordinates for each country

In [15]:
coords_raw = pd.read_csv("coords_country.csv")
coords_raw.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [16]:
#To cut longitude and latitude to 3 decimals
coords_raw["longitude"] = ["{:.3f}".format(lng) for lng in coords_raw["longitude"]]
coords_raw["latitude"] = ["{:.3f}".format(lat) for lat in coords_raw["latitude"]]

In [17]:
#Remove "country" column
coords_raw = coords_raw.drop(["country"], axis = 1)

In [18]:
#Change columns names
coords_raw.rename(columns = {"latitude" : "Latitude", "longitude" : "Longitude"}, inplace = True)

In [19]:
#2nd merge: air_poll and coords_raw
complete_air_poll = air_poll.merge(coords_raw, left_on = ["Country"], right_on = ["name"])
#Remove "name column"
complete_air_poll = complete_air_poll.drop(["name"], axis = 1)

### Final dataframe

In [20]:
complete_air_poll

,Country,Year,Mean exposure toPM2.5 (µg/m3),"Deaths per 100,000",Ratified Kyoto P.,Latitude,Longitude
0,Australia,1990,10.36402,26.705027,True,-25.274,133.775
1,Australia,2010,10.53559,13.571713,True,-25.274,133.775
2,Australia,2011,10.88267,13.727626,True,-25.274,133.775
3,Australia,2012,10.40148,12.659725,True,-25.274,133.775
4,Australia,2013,9.84329,11.874490,True,-25.274,133.775
...,...,...,...,...,...,...,...
1291,Iran,2012,38.80363,46.800381,True,32.428,53.688
1292,Iran,2013,38.42392,46.328081,True,32.428,53.688
1293,Iran,2014,38.27532,46.446436,True,32.428,53.688
1294,Iran,2015,39.97714,47.209842,True,32.428,53.688


In [21]:
g7 = ["Canada", "France", "Germany", "Italy", "Japan", "United Kingdom", "United States"]

g20 = ["Argentina", "Australia", "Brazil", "Canada", "China", "France", "Germany", "India", "Indonesia", "Italy", "Japan",
      "Mexico", "Russia", "Saudi Arabia", "South Africa", "South Korea", "Turkey", "United Kingdom", "United States"]

## --- Ignore --- 

In [22]:
air_poll.loc[air_poll["Country"].str.contains("United")]

,Country,Year,Mean exposure toPM2.5 (µg/m3),"Deaths per 100,000",Ratified Kyoto P.
200,United Kingdom,1990,13.24545,42.699369,True
201,United Kingdom,2010,12.30340,21.117271,True
202,United Kingdom,2011,12.84189,20.822217,True
203,United Kingdom,2012,11.80623,19.487839,True
204,United Kingdom,2013,11.60548,19.289088,True
205,United Kingdom,2014,10.78696,18.084411,True
206,United Kingdom,2015,10.73807,18.243927,True
207,United Kingdom,2016,10.48657,17.891337,True
208,United States,1990,9.56131,31.195070,False
209,United States,2010,9.29297,22.972047,False


In [24]:
no_rat = air_poll.loc[air_poll["Ratified Kyoto P."] == False]

In [25]:
no_rat["Country"].value_counts()

Chad             8
Afghanistan      8
Zimbabwe         8
Andorra          8
Somalia          8
United States    8
Moldova          8
Kazakhstan       8
Iraq             8
Name: Country, dtype: int64